In [1]:
!pip install gymnasium renderlab
!pip install opencv-python
!pip install pygame

import gymnasium as gym

# Custom Frozen Lake maze layout
desc = ["SFFF",
        "FHHH",
        "FFFF",
        "HFHF",
        "FFGF"]

# Initialize the environment
env = gym.make('FrozenLake-v1', desc=desc, map_name="5x5", is_slippery=False)
observation, info = env.reset()

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [13]:
import numpy as np
import time
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def monte_carlo_policy_evaluation(env, num_episodes, gamma=0.9):
    # Initialize value function arbitrarily
    V = np.zeros(env.observation_space.n)
    
    # Initialize count of visits to states
    N = np.zeros(env.observation_space.n)
    
    statistics = pd.DataFrame({
        "Time": [],
        "Episode": [],
        "GotGift": []
    })
    
    start_time = time.time()
    
    # Loop over episodes
    for episode in range(num_episodes):
        state = env.reset()
        episode_states = []
        episode_rewards = []
        done = False
        
        # Generate an episode
        while not done:
            # Take random action
            action = env.action_space.sample()
            next_state, reward, done, _, _ = env.step(action)
            episode_states.append(state)
            episode_rewards.append(reward)
            state = next_state
        
        # Calculate returns
        G = 0
        for t in reversed(range(len(episode_states))):
            G = gamma * G + episode_rewards[t]
            state = episode_states[t]
            if state not in episode_states[:t]:
                N[state] += 1
                V[state] += (1 / N[state]) * (G - V[state])
        
        # Check if the episode reached the goal state
        got_gift = 1 if reward == 1 else 0
        
        end_time = time.time()
        elapsed_time = end_time - start_time
        
        statistics.loc[episode] = {
            "Time": elapsed_time,
            "Episode": episode + 1,
            "GotGift": got_gift
        }
        
        # Print progress
        if (episode + 1) % 100 == 0:
            print(f"Episode {episode + 1}/{num_episodes} completed")

    return V, statistics

# Create the environment
desc = ["SFFF", "FHHH", "FFFF", "HFHF", "FFGF"]
env = gym.make('FrozenLake-v1', desc=desc, map_name="5x5", is_slippery=False) 

# Define parameters
num_episodes = 5000

# Perform Monte Carlo policy evaluation
V, statistics = monte_carlo_policy_evaluation(env, num_episodes)

# Optimal policy
optimal_policy = np.argmax(V, axis=1)
print("Optimal policy:")
print(optimal_policy)

# Create the scatter plot
sns.scatterplot(data=statistics, x="Time", y="GotGift", hue="GotGift")

# Set the limits of x and y axes
plt.ylim(-0.1, 1.1)

# Show the plot
plt.show()

# Save statistics to a CSV file
statistics.to_csv("monte_carlo_statistics.csv")

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices